In [4]:
# Set up
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
gpd.io.file.fiona.drvsupport.supported_drivers['kml'] = 'rw'

NameError: name 'gpd' is not defined

In [3]:
conda install geopandas

Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: ...working... unsuccessful attempt using repodata from current_repodata.json, retrying with next repodata source.
Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
failed

Note: you may need to restart the kernel to use updated packages.



Examining @/win-64::__cuda==12.2=0:  80%|████████  | 4/5 [00:00<00:00, 11.81it/s]      
                                                                                 

Examining conflict for python geopandas:   0%|          | 0/5 [00:00<?, ?it/s]
                                                                              

UnsatisfiableError: The following specifications were found to be incompatible with each other:

Output in format: Requested package -> Available versions



: 

In [2]:
import geopandas
import pandas as pd
from here_plolocation_services import LS
from shapely.geometry import Point
import numpy as np
from sklearn.cluster import KMeans
from shapely.geometry import Point
from shapely.geometry import LineString
from shapely.ops import transform
import numpy as np
import networkx as nx

ModuleNotFoundError: No module named 'geopandas'

In [59]:
# query for state and MN railroad
rail = gpd.read_file(
    "https://geo.dot.gov/mapping/rest/services/NTAD/North_American_Rail_Network_Lines/MapServer/0/query?where=STATEAB%20%3D%20'MN'&text=&objectIds=&time=&timeRelation=esriTimeRelationOverlaps&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&returnExtentOnly=false&sqlFormat=none&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=pjson"
).set_index("YARDNAME")

In [95]:
# get length of rail
rail['length'] = rail['geometry'].apply(lambda geom: geom.length)

In [97]:
# get shape of route, find end and beginning coordinates
# get coordinates of rail for origin/destination

def coord_lister(geom):
    if geom.geom_type == 'MultiLineString':
        # Extract coordinates of start and end points for each LineString in the MultiLineString
        coords = [list(line.coords[0]) + list(line.coords[-1]) for line in geom]
    elif geom.geom_type == 'LineString':
        # For a single LineString, extract its start and end coordinates
        coords = list(geom.coords[0]) + list(geom.coords[-1])
    return coords

# Apply the modified function to  GeoDataFrame
coordinates = rail.geometry.apply(coord_lister)

In [104]:
def xingCloseRoute(name, point, offset, buffer, rail):
    # Create a GeoSeries for the input point
    point_gdf = gpd.GeoSeries(Point(point))

    # Buffer the point and create a LineString
    buffered_line = point_gdf.buffer(buffer).iloc[0]
    buffered_line = buffered_line.boundary  # Extract the boundary as a LineString

    # Ensure both GeoSeries have the same index
    buffered_line = gpd.GeoSeries([buffered_line], index=[0])
    rail = rail.set_index(range(len(rail)))

    # Perform the intersection with the rail dataset
    intersection = buffered_line.intersection(rail.geometry)
    intersection = intersection[~intersection.is_empty]

    # Convert the resulting geometry to Points
    intersection_points = intersection[intersection.type == 'Point']
    
    # Get the coordinates
    coords = np.array(list(intersection_points.apply(coord_lister)))
    
    # Perform k-means clustering
    centers, _ = kmeans(coords, 2)

    # Calculate cluster statistics
    xavg = np.mean(centers[:, 0])
    yavg = np.mean(centers[:, 1])
    xdiff = centers[0, 0] - centers[1, 0]
    ydiff = centers[0, 1] - centers[1, 1]
    
    # Calculate the angle
    angle = np.arctan2(ydiff, xdiff) + np.pi / 2
    
    # Calculate new coordinates
    x1 = xavg + np.cos(angle) * offset
    y1 = yavg + np.sin(angle) * offset
    x2 = xavg - np.cos(angle) * offset
    y2 = yavg - np.sin(angle) * offset
    
    # Create a GeoDataFrame with the results
    side1 = gpd.GeoDataFrame({'point': ['side1'], 'geometry': [Point(x1, y1)]})
    side2 = gpd.GeoDataFrame({'point': ['side2'], 'geometry': [Point(x2, y2)]})
    navPoints = gpd.GeoDataFrame(pd.concat([side1, side2]))
    
    # Create a barrier by buffering the point, converting to LineString, and buffering
    barrier = point_gdf.buffer(buffer).boundary.iloc[0]
    barrier = barrier.buffer(10)

    # Create a network graph to find the route
    G = nx.Graph()
    G.add_node('side1', geometry=navPoints[navPoints['point'] == 'side1']['geometry'].iloc[0])
    G.add_node('side2', geometry=navPoints[navPoints['point'] == 'side2']['geometry'].iloc[0])
    G.add_edge('side1', 'side2', weight=1)  # Add a dummy edge with weight 1
    
    # Find the route using NetworkX
    route_result = nx.shortest_path(G, source='side1', target='side2', weight='weight')

    # Create a GeoDataFrame for the route
    route_gdf = gpd.GeoDataFrame({'name': [name], 'geometry': LineString(
        [navPoints['geometry'][route_result[0]], 
         navPoints['geometry'][route_result[1]]])})

    return route_gdf

In [105]:
# Example usage
name = "your_route_name"
point = (-92.60064662057485, 47.45905019541532)
offset = 100
buffer = 0.01
rail = rail

result_gdf = xingCloseRoute(name, point, offset, buffer, rail)
print(result_gdf)

KeyError: 'None of [range(0, 2000)] are in the columns'

In [60]:
def xingCloseRoute(name, point, offset, buffer, rail):
    # Create a Point from the input coordinates
    point = Point(point)


    # Buffer the point and convert to a LineString
    buffered_line = point.buffer(buffer).boundary

    # Ensure both GeoSeries have the same index
    buffered_line_gdf = gpd.GeoSeries([buffered_line], index=[0])
    rail = rail.set_index(range(len(rail)))  # Reset the index to match the new GeoSeries

    # Perform the intersection with the rail dataset
    intersection = gpd.GeoSeries([buffered_line]).intersection(rail)

    # Convert the resulting geometry to Points
    intersection_points = intersection[intersection.type == 'Point']

    # Get the coordinates
    coords = np.array(list(intersection_points.apply(lambda geom: geom.coords[0])))

    # Perform k-means clustering
    centers, _ = kmeans(coords, 2)

    # Calculate cluster statistics
    xavg = np.mean(centers[:, 0])
    yavg = np.mean(centers[:, 1])
    xdiff = centers[0, 0] - centers[1, 0]
    ydiff = centers[0, 1] - centers[1, 1]

    # Calculate the angle
    angle = np.arctan2(ydiff, xdiff) + np.pi / 2

    # Calculate new coordinates
    x1 = xavg + np.cos(angle) * offset
    y1 = yavg + np.sin(angle) * offset
    x2 = xavg - np.cos(angle) * offset
    y2 = yavg - np.sin(angle) * offset

    # Create a DataFrame with the results
    df = gpd.GeoDataFrame({'point': ['side1', 'side2'],
                           'xnew': [x1, x2],
                           'ynew': [y1, y2]})
    return df

# Example usage
name = "your_route_name"
point = (-92.60064662057485, 47.45905019541532)
offset = 100  # Your desired offset value
buffer = 0.01  # Your desired buffer value
rail = rail # Load your rail dataset

result_df = xingCloseRoute(name, point, offset, buffer, rail)
print(result_df)

KeyError: 'None of [range(0, 2000)] are in the columns'

In [61]:
length(rail)

NameError: name 'length' is not defined

In [ ]:
# Function to create points on each side of the track and find route ---------------------------------------------------------

xingCloseRoute <- function(name, point, offset, buffer, rail) { # name of route, point is location for start of route, offset from point, bufffer, rail the actual dataset
  navPoints <- point %>% st_buffer(buffer) # creaste code from buffer along buffer distanace
                     %>% st_cast("LINESTRING") %>% st_intersection(rail) %>% st_cast("POINT") %>%  # converts to linestring, 

In [6]:
# Function to calculate navigation points
def calculate_navigation_points(point, buffer, rail):
    navPoints = point.buffer(buffer).intersection(rail).centroid
    return pd.DataFrame({'x': navPoints.x, 'y': navPoints.y})

In [7]:
# Function to perform K-means clustering
def perform_kmeans_clustering(navPoints):
    kmeans = KMeans(n_clusters=2, random_state=0).fit(navPoints)
    centers = kmeans.cluster_centers_
    return centers

In [8]:
# Function to calculate angle and new coordinates
def calculate_angle_and_coordinates(centers, offset):
    xavg, yavg = centers[0]
    xdiff, ydiff = centers[0] - centers[1]
    angle = np.arctan(ydiff / xdiff) + np.pi / 2

    x1 = xavg + np.cos(angle) * offset
    y1 = yavg + np.sin(angle) * offset
    x2 = xavg - np.cos(angle) * offset
    y2 = yavg - np.sin(angle) * offset

    return x1, y1, x2, y2

In [9]:
# Function to create navigation points DataFrame
def create_navpoints_dataframe(x1, y1, x2, y2):
    navPoints = pd.DataFrame({
        'point': ["side1", "side2"],
        'xnew': [x1, x2],
        'ynew': [y1, y2]
    })
    return navPoints

In [10]:
# Function to create a "barrier"
def create_barrier(point, buffer, rail):
    barrier = point.buffer(buffer).intersection(rail).envelope.buffer(10)
    return barrier

In [28]:
create_barrier(point=point,)

TypeError: create_barrier() missing 2 required positional arguments: 'buffer' and 'rail'

In [11]:
# Function to calculate routes
def calculate_routes(navPoints, barrier):
    routeTest1 = nearest_nodes(graph, navPoints[navPoints['point'] == "side1"]['xnew'].values[0], navPoints[navPoints['point'] == "side1"]['ynew'].values[0])
    routeTest2 = nearest_nodes(graph, navPoints[navPoints['point'] == "side2"]['xnew'].values[0], navPoints[navPoints['point'] == "side2"]['ynew'].values[0])
    return routeTest1, routeTest2

In [25]:
# Function to create points on each side of the track and find route
def xingCloseRoute(name, point, offset, buffer, rail):
    navPoints = point.buffer(buffer).intersection(rail).centroid
    navPoints = pd.DataFrame({'x': navPoints.x, 'y': navPoints.y})
    
    # K-means clustering
    kmeans = KMeans(n_clusters=2, random_state=0).fit(navPoints)
    centers = kmeans.cluster_centers_
    
    # Calculate angle and new coordinates
    xavg, yavg = centers[0]
    xdiff, ydiff = centers[0] - centers[1]
    angle = np.arctan(ydiff / xdiff) + np.pi / 2

    x1 = xavg + np.cos(angle) * offset
    y1 = yavg + np.sin(angle) * offset
    x2 = xavg - np.cos(angle) * offset
    y2 = yavg - np.sin(angle) * offset

    navPoints['point'] = ["side1", "side2"]
    navPoints['xnew'] = [x1, x2]
    navPoints['ynew'] = [y1, y2]

    # Create a "barrier"
    barrier = point.buffer(buffer).intersection(rail).envelope.buffer(10)

    # Calculate routes
    routeTest1 = ls.route(
        origin=(navPoints[navPoints['point'] == "side1"]['xnew'].values[0], navPoints[navPoints['point'] == "side1"]['ynew'].values[0]),
        destination=(navPoints[navPoints['point'] == "side2"]['xnew'].values[0], navPoints[navPoints['point'] == "side2"]['ynew'].values[0]),
        mode='car',
        avoid=[barrier]
    )

    routeTest2 = ls.route(
        origin=(navPoints[navPoints['point'] == "side2"]['xnew'].values[0], navPoints[navPoints['point'] == "side2"]['ynew'].values[0]),
        destination=(navPoints[navPoints['point'] == "side1"]['xnew'].values[0], navPoints[navPoints['point'] == "side1"]['ynew'].values[0]),
        mode='car',
        avoid=[barrier]
    )

    return routeTest1, routeTest2

In [26]:
# Calculate the routes using xingCloseRoute function
test1, test2 = xingCloseRoute("test run", "9", offset=200, buffer=15, rail=graph)

NameError: name 'graph' is not defined

In [23]:
# Create a Point geometry
point = Point(-92.60064662057485, 47.45905019541532)

# Create a GeoDataFrame
p = gpd.GeoDataFrame({'id': ['barrier'], 'geometry': [point]}, crs="EPSG:4326")



# Calculate the routes using xingCloseRoute function
test1, test2 = xingCloseRoute("test run", barrier, offset=200, buffer=15, rail=graph)

# Calculate the routes
routeTest1, routeTest2 = xingCloseRoute("test run", p, 200, 15, rail)

# Print the routes
print(routeTest1)
print(routeTest2)

NameError: name 'barrier' is not defined

#### Map--------------------

In [ ]:
# Plot the routes
fig, ax = ox.plot_graph_routes(graph, [test1, test2], route_colors=['darkred', 'darkblue'],
                               route_linewidth=5, node_size=0, orig_dest_size=50, save=False, close=False)

# Define basemap options
basemap = ox.basemap_to_gdf(zoom=11, bmap_type=['CartoDB.Positron', 'OpenStreetMap.Mapnik', 'Esri.WorldImagery'])

# Plot the basemap
basemap.plot(ax=ax, alpha=0.5)

In [ ]:
# Create the time based isolines
barrier_centroid = barrier['geometry'].centroid.iloc[0]
isoTest = ox.isolines.route_isolines(barrier_centroid, network=graph, travel_speed=20,
                                     travel_mode='car', intervals=[5, 10, 15, 20], output_type='geoDataFrame')

# Calculate and add minutes column
isoTest['minutes'] = (isoTest['geometry'].length / 1000) / 20 * 60

# Validate isolines
isoTest['is_valid'] = isoTest.is_valid

# Plot isolines and points
fig, ax = ox.plot_graph_routes(graph, isoTest, route_colors='inferno',
                               route_linewidth=5, node_size=0, orig_dest_size=50, ax=ax)

# Plot points
points = barrier.to_crs('EPSG:4326')
points.plot(ax=ax, marker='o', color='purple', markersize=50)

plt.show()